In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator, partial_trace
import matplotlib.pyplot as plt
from typing import Callable, Dict, Iterable, Union


## Introduction
In classical computing, a **Read-Only Memory (ROM)** stores fixed data that can be queried but not modified.  
A **Quantum ROM (QROM)** provides a quantum analogue: it stores a classical function $f(x)$ so that querying it acts coherently on a superposition of inputs.

Formally, for $n$ input qubits and $d$ output qubits, the QROM unitary satisfies

$$
U_f\lvert x\rangle_n\lvert 0\rangle_d = \lvert x\rangle_n\lvert f(x)\rangle_d.
$$

This enables efficient oracle construction in quantum algorithms such as Grover search, quantum machine learning, and amplitude encoding.

## Mathematical Background
A Boolean function $f:\mathbb F_2^n\mapsto\mathbb F_2^d$ maps $n$-bit strings to $d$-bit outputs.  
Each bit of $f(x)$ can be implemented as a reversible map using **multi-controlled X (Toffoli) gates**:

$$
x \mapsto (x,\,y\oplus f(x)).
$$

If $f(x)$ is known classically, the QROM construction consists of:
1. Building the truth table of $f(x)$.
2. Adding controlled-NOTs to set each output qubit according to the pattern of control bits that match $x$.
3. Optionally optimizing the decomposition using ancilla or binary-tree lookup methods.

### Implementation Steps

1. **Determine qubit counts:**  
   Compute the number of input and output qubits: $n$, number of input qubits and $d$, number of output qubits
2. **Create the quantum circuit:**
3. **Loop over entries in f_table:**
   For each mapping $x\mapsto f(x)$:
   Apply a multi-controlled X gate on each output bit $i$ where $f_{i}(x)=1$
4. **Uncompute ancilla** (if any):
   If ancillary qubits are used during computation, apply the necessary gates to uncompute them, ensuring the circuit remains reversible.
 


In [16]:
def int_to_bitlist(x: int, n: int) -> list:
    return [int(b) for b in format(x, f'0{n}b')]  #Return list of bits (MSB first) for integer x, zero-padded to length n.#


def build_quantum_rom(f: Dict[int, Union[str, Iterable[int]]],n: int,d: int,name: str = "QROM") -> QuantumCircuit:
    
                                                  #Build a quantum circuit implementing ROM for function f: {0,1}^n -> {0,1}^d.#
                                                  #Returns a circuit that maps |x>|0> -> |x>|f(x)>.#

    inp = QuantumRegister(n, "Input")
    out = QuantumRegister(d, "Output")
    anc = AncillaRegister(max(0, n - 2), "Anc") if n > 2 else None



    
    regs = [inp, out] + ([anc] if anc else [])
    qc = QuantumCircuit(*regs, name=name)

    for x in range(2 ** n):
        fx = f.get(x, "0" * d)
        fx_bits = [int(b) for b in fx]

        if all(b == 0 for b in fx_bits):           # skip if all outputs are 0#
            continue

        x_bits = int_to_bitlist(x, n)
        flip_qubits = [i for i, bit in enumerate(x_bits) if bit == 0]

        for i in flip_qubits:                      # Flip controls where we need to control on 0#
            qc.x(inp[i])

        for j, b in enumerate(fx_bits):            # Apply controlled Xs for outputs that should be 1#
            if b == 1:
                if n == 1:
                    qc.cx(inp[0], out[j])
                elif n == 0:
                    qc.x(out[j])
                else:
                    qc.mcx(inp, out[j], anc, mode='v-chain' if anc else 'noancilla')

        for i in flip_qubits:                      # Undo the flips
            qc.x(inp[i])

    return qc

## Demonstration for n = 3

Example function $f(x_{0}x_{1}x_{1})=x_{0}\oplus x_{1}x_{2}$ on one-bit output $d=1$:

This circuit performs the reversible lookup in superposition:

$$
\frac{1}{\sqrt{8}} \sum_x \lvert x \rangle \lvert 0 \rangle 
\;\xrightarrow{U_f}\;
\frac{1}{\sqrt{8}} \sum_x \lvert x \rangle \lvert f(x) \rangle.
$$

In [23]:
f_table = {                                       # Define a small truth table f: {0,1}^3 → {0,1}^2
    0: "00",
    1: "01",
    2: "10",                                      # Example mapping (you can change this freely)
    3: "11",
    4: "01",
    5: "10",
    6: "11",
    7: "00"
}

n, d = 3, 2
qrom = build_quantum_rom(f_table, n, d)
print("=== Quantum ROM Circuit ===")
print(qrom.draw(fold=120))

=== Quantum ROM Circuit ===
          ┌───┐      ┌───┐┌───┐      ┌───┐┌───┐            ┌───┐                                            
 Input_0: ┤ X ├──■───┤ X ├┤ X ├──■───┤ X ├┤ X ├──■─────■───┤ X ├──■───────────────■──────────■─────■────────
          ├───┤  │   ├───┤└───┘  │   └───┘└───┘  │     │   ├───┤  │   ┌───┐┌───┐  │   ┌───┐  │     │        
 Input_1: ┤ X ├──■───┤ X ├───────■───────────────■─────■───┤ X ├──■───┤ X ├┤ X ├──■───┤ X ├──■─────■────────
          └───┘  │   ├───┤       │   ┌───┐       │     │   ├───┤  │   ├───┤└───┘  │   ├───┤  │     │   ┌───┐
 Input_2: ───────■───┤ X ├───────■───┤ X ├───────■─────■───┤ X ├──■───┤ X ├───────■───┤ X ├──■─────■───┤ X ├
                 │   └───┘     ┌─┴──┐└───┘     ┌─┴──┐  │   └───┘  │   └───┘     ┌─┴──┐└───┘┌─┴──┐  │   └───┘
Output_0: ───────┼─────────────┤0   ├──────────┤0   ├──┼──────────┼─────────────┤0   ├─────┤0   ├──┼────────
               ┌─┴──┐          │    │          │    │┌─┴──┐     ┌─┴──┐          │    │     │    │┌─┴

/tmp/ipykernel_5776/2181959812.py:40: DeprecationWarning: ``qiskit.circuit.quantumcircuit.QuantumCircuit.mcx()``'s argument ``mode`` is deprecated as of Qiskit 2.1. It will be removed no earlier than 3 months after the release date. Instead, add a generic MCXGate to the circuit and specify the synthesis method via the ``hls_config`` in the transpilation. Alternatively, specific decompositions are available at https://qisk.it/mcx.
  qc.mcx(inp, out[j], anc, mode='v-chain' if anc else 'noancilla')
/tmp/ipykernel_5776/2181959812.py:40: DeprecationWarning: ``qiskit.circuit.quantumcircuit.QuantumCircuit.mcx()``'s argument ``mode`` is deprecated as of Qiskit 2.1. It will be removed no earlier than 3 months after the release date. Instead, add a generic MCXGate to the circuit and specify the synthesis method via the ``hls_config`` in the transpilation. Alternatively, specific decompositions are available at https://qisk.it/mcx.
  qc.mcx(inp, out[j], anc, mode='v-chain' if anc else 'noancilla'

## Testing and Verification

To verify the correctness of the Quantum ROM circuit:

1. Prepare all computational-basis input states.
2. Apply the constructed unitary $U_{f}$.
3. Measure the outputs and compare them with the classical function $f(x)$.


### Expected Result:
Each input state $\lvert x \rangle$ should produce a corresponding output state $\lvert f(x) \rangle$, confirming the correct reversible mapping of the Quantum ROM.

In [25]:
has_anc = any(reg.name == "Anc" for reg in qrom.qregs)
anc_size = next((reg.size for reg in qrom.qregs if reg.name == "Anc"), 0)

for x in range(2 ** n):
    # Build a prep circuit with the same registers as qrom
    inp = QuantumRegister(n, "Input")
    out = QuantumRegister(d, "Output")
    anc = AncillaRegister(anc_size, "Anc") if anc_size > 0 else None
    regs = [inp, out] + ([anc] if anc else [])
    qc_prep = QuantumCircuit(*regs)

    bits = format(x, f'0{n}b')
    for i, b in enumerate(bits):
        if b == '1':
            qc_prep.x(inp[i])

    # Combine with the QROM
    qc_full = qc_prep.compose(qrom)

    # Compute final statevector (no backend, no AER)
    sv = Statevector.from_instruction(qc_full)

    probs = sv.probabilities_dict()
    outcome, p = max(probs.items(), key=lambda kv: kv[1])

    print(f"x={x}: output bitstring = {outcome} (prob={p:.3f})")

x=0: output bitstring = 000000 (prob=1.000)
x=1: output bitstring = 010100 (prob=1.000)
x=2: output bitstring = 001010 (prob=1.000)
x=3: output bitstring = 011110 (prob=1.000)
x=4: output bitstring = 010001 (prob=1.000)
x=5: output bitstring = 001101 (prob=1.000)
x=6: output bitstring = 011011 (prob=1.000)
x=7: output bitstring = 000111 (prob=1.000)


## Optimizations and Discussion

### Possible Optimizations

- **Tree-based QROM:**  
  This method reduces the gate count from $ O(2^n) $ to roughly $ O(2^n / n) $ by reusing ancilla qubits in a binary tree structure for data loading.

- **QROM via Quantum Fourier Transform (QFT):**  
  For arithmetic or phase-related functions $f(x)$, controlled phase rotations or modular addition circuits can substitute for multi-controlled NOT networks, often reducing circuit depth.

- **Gate Decomposition and Ancilla Usage:**  
  Multi-controlled gates (e.g., MCX) can be decomposed into Toffoli and single-qubit gates, using ancillas to optimize both the depth and total gate count for hardware execution.

### Practical Considerations

Quantum ROMs are essential in:
- **Grover’s Search** – serving as the oracle for marked states.  
- **Quantum Machine Learning** – encoding classical data into quantum amplitudes.  
- **Quantum Arithmetic and Simulation** – embedding lookup tables for constants or precomputed values.

**Challenges:**
- Large control registers cause exponential gate scaling.  
- Hardware noise and qubit connectivity limit implementation fidelity.  
- Efficient decompositions and approximate oracles are often required for real devices.

---

## Conclusion

This project illustrates the construction of a **Quantum ROM (QROM)** circuit implementing a reversible, coherent lookup of a Boolean function $f(x)$.  
By designing the unitary $U_{f}$ such that

$$
U_f \lvert x \rangle \lvert 0 \rangle = \lvert x \rangle \lvert f(x) \rangle,
$$

we demonstrated how classical data access can be extended into quantum superposition, forming a building block for many quantum algorithms.

### Key Insights
- QROM acts as a **quantum oracle**, enabling simultaneous queries over all inputs.  
- The construction showcases the importance of **reversibility** and **coherence** in quantum data access.  
- Optimizations such as **tree-based QROM** or **bucket-brigade architectures** are vital for scalability.

### Future Directions
- Implement compressed QROM designs to reduce hardware cost.  
- Benchmark circuit depth and fidelity on real IBM Quantum hardware.  
- Integrate the QROM within larger quantum algorithms such as **Grover’s Search** and **Quantum Phase Estimation**.

---
